# Paper Savior with LionAGI and LlamaIndex Vector Index

-- how to do auto explorative research with LionAGI plus RAG using llamaindex Vector Index & embedding 

- [LionAGI](https://github.com/lion-agi/lionagi)
- [LlamaIndex](https://www.llamaindex.ai)

In [5]:
# %pip install lionagi llama_index

In [6]:
query = 'Large Language Model Time Series Analysis'
dir = "data/log/researcher/"
num_papers = 5

### 1. Build a Vector Index with llama_index

In [7]:
from llama_index import download_loader, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.node_parser import SentenceSplitter

In [8]:
ArxivReader = download_loader("ArxivReader")
loader = ArxivReader()
node_parser = SentenceSplitter(chunk_size=800, chunk_overlap=50)

# let us download some papers from arvix
documents, abstracts = loader.load_papers_and_abstracts(search_query=query, max_results=num_papers)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=False)

# set up index object
llm = OpenAI(temperature=0.1, model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)
index1 = VectorStoreIndex(nodes, include_embeddings=True, service_context=service_context)

# set up query engine
query_engine = index1.as_query_engine(include_text=False, response_mode="tree_summarize")

In [9]:
len(abstracts)

5

### 2. Write a tool description according to OpenAI schema

In [10]:
import lionagi as li

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_arvix_papers",
            "description": "Perform a query to a QA bot with access to a knowledge graph index built with papers from arvix",
            "parameters": {
                "type": "object",
                "properties": {
                    "str_or_query_bundle": {
                        "type": "string",
                        "description": "a question to ask the QA bot",
                    }
                },
                "required": ["str_or_query_bundle"],
            },
        }
    }
]

func = query_engine.query

In [12]:
system = {
    "persona": "a helpful world-class researcher",
    "requirements": "think step by step before returning a clear, precise worded answer with a humble yet confident tone",
    "responsibilities": f"you are asked to help with researching on the topic of {query}",
    "tools": "provided with a QA bot for grounding responses"
}

### 3. Research: PROMPTS

#### FORMATS

In [13]:
deliver_format1 = {"return required": "yes", "return format": "paragraph"}

deliver_format2 = {"return required": "yes", 
    "return format": {
        "type": "json_mode", 
        "format": {
            'paper': "paper_name",
            "summary": "...", 
            "research question": "...", 
            "talking points": {
                "point 1": "...",
                "point 2": "...",
                "point 3": "..."
            }}}},

deliver_format3 = {
    "notice":"when entering each task step, identified by step number, if it is the first time that instruction is given,  you must use the tool at least once. Notice you are provided with a QA bot as your tool, the bot has access to the papers via a queriable index that takes natural language query and return a natural language answer. You can decide whether to invoke the function call, you will need to ask the bot when there are things need clarification or further information. you provide the query by asking a question, please use the tool extensively as you can", 
    "tool choice": "auto", 
    "format":"function calling"}

#### PROMPTS

In [14]:
instruct1 = {
    "task step": "1", 
    "task name": "read paper abstracts", 
    "task objective": "get initial understanding of the papers of interest", 
    "task description": "provided with abstracts of paper, provide a brief summary highlighting the paper core points, the purpose is to extract as much information as possible",
    "deliverable": deliver_format1
}

instruct2 = {
    "task step": "2",
    "task name": "propose research questions and talking points", 
    "task objective": "initial brainstorming", 
    "task description": "from the improved understanding of the paper, please propose an interesting, unique and practical research questions, support your reasoning. Kept on asking questions if things are not clear",
    "deliverable": deliver_format2,
    "function calling": deliver_format3
}

### 4. Research: Setup Workflow

In [15]:
abstracts = li.l_call(range(len(abstracts)), lambda i: abstracts[i].text)

In [16]:
async def read_propose(context, num=5):
    researcher = li.Session(system, dir=dir)
    researcher.register_tools(tools, func)
    
    await researcher.initiate(instruct1, context=context, temperature=0.7)
    await researcher.auto_followup(instruct2, tools=tools, num=num, tool_parser=lambda x: x.response)
    
    researcher.messages_to_csv()
    researcher.log_to_csv()
    return researcher

### 5. Research: Run the workflow

In [17]:
researchers = await li.al_call(abstracts, read_propose)

8 logs saved to data/log/researcher/_messages_2023-12-14T21_27_32_206873.csv
3 logs saved to data/log/researcher/_llmlog_2023-12-14T21_27_32_207702.csv
8 logs saved to data/log/researcher/_messages_2023-12-14T21_27_32_311150.csv
3 logs saved to data/log/researcher/_llmlog_2023-12-14T21_27_32_311672.csv
8 logs saved to data/log/researcher/_messages_2023-12-14T21_27_52_174436.csv
3 logs saved to data/log/researcher/_llmlog_2023-12-14T21_27_52_175080.csv
8 logs saved to data/log/researcher/_messages_2023-12-14T21_27_54_939203.csv
3 logs saved to data/log/researcher/_llmlog_2023-12-14T21_27_54_940608.csv
8 logs saved to data/log/researcher/_messages_2023-12-14T21_28_02_312351.csv
3 logs saved to data/log/researcher/_llmlog_2023-12-14T21_28_02_313009.csv


In [18]:
out1 = li.l_call(researchers, lambda x: x.conversation.messages[-1]['content'])

In [20]:
Markdown(out1[0])

Based on the understanding of the paper and the potential limitations identified for LPTM, here is a proposed research question along with supporting talking points:

```json
{
  "paper": "Large Pre-trained Time Series Models for Cross-domain Time Series Analysis Tasks",
  "summary": "The paper introduces a novel approach to time series analysis by developing a Large Pre-trained Time Series Model (LPTM) that can be applied across different domains. LPTM aims to be efficient in terms of data and computational resources, potentially reducing the amount of data and training time required to achieve state-of-the-art performance.",
  "research question": "How can LPTM be adapted to maintain high performance when dealing with real-world time series data that is noisy, incomplete, or non-representative?",
  "talking points": {
    "point 1": "Investigating methods for noise filtering and data imputation within the LPTM framework to enhance its robustness to poor quality data.",
    "point 2": "Exploring the potential of transfer learning techniques within LPTM to better adapt to specific real-world domains where data may be scarce or different in nature.",
    "point 3": "Assessing the computational efficiency of LPTM in real-time applications and identifying optimization strategies for reducing latency without compromising performance."
  }
}
```

These research questions and talking points delve into practical aspects of deploying LPTM in real-world settings, where data quality and computational constraints are common issues. The aim is to examine strategies to further enhance the adaptability and efficiency of LPTM to ensure it can be effectively utilized in a variety of practical applications.

In [21]:
Markdown(out1[1])

Based on the information provided about Time-LLM's approach to handling data sparsity in time series forecasting through the reprogramming of large language models, I propose the following research question and talking points:

**Research Question:** 
How can the reprogramming approach used by Time-LLM be extended to incorporate external domain-specific knowledge for improved forecast accuracy in highly specialized fields like finance or meteorology?

**Talking Points:**
- **Point 1:** Investigate the integration of domain-specific ontologies or knowledge graphs with Time-LLM to enhance its few-shot learning capabilities further and tailor its reasoning to industry-specific forecasting tasks.

- **Point 2:** Explore the potential for Time-LLM to adapt to real-time data streams in dynamic environments, assessing the model's responsiveness and accuracy in rapidly changing conditions.

- **Point 3:** Examine the transferability of Time-LLM's reprogramming framework to other types of structured data beyond time series, such as spatial or graph-based data, to understand the versatility and limitations of this approach.

These talking points are designed to expand the conversation around Time-LLM's innovative approach to time series forecasting, seeking ways to optimize and broaden its application across different domains and data modalities.

Should further clarification be required on any of these points, we can utilize the QA bot to query the necessary information.

In [22]:
Markdown(out1[2])

Based on the provided summary and the practical applications of the Frozen Pretrained Transformer (FPT), an interesting research question could delve into the adaptability and transfer learning capabilities of FPT across diverse domains. It would be practical to explore how the model performs when transitioning between industries that have fundamentally different types of time series data and what fine-tuning strategies are most effective.

Research Question: How can the Frozen Pretrained Transformer (FPT) be adapted for optimal performance across diverse industries with unique time series data characteristics, and what are the best practices for fine-tuning the model in these varying contexts?

Talking Points:
- **Point 1**: Investigate the impact of industry-specific data features on the performance of FPT. For example, how does the inherent noise in financial time series compare to the periodicity in energy consumption data, and what implications does this have for model fine-tuning and generalization?
- **Point 2**: Examine the fine-tuning process of FPT when applied to industries with limited data availability. This includes exploring techniques like few-shot learning or data augmentation to enhance model performance where large datasets are not available.
- **Point 3**: Assess the theoretical underpinnings of FPT's adaptability by studying how the model's self-attention mechanism compares to PCA in different data contexts. Understanding this relationship may provide insights into how to better leverage the model's capabilities for specific industry applications.

These talking points aim to push the boundaries of understanding how FPT can be fine-tuned and applied effectively across various industries, ensuring that practitioners can harness its full potential for their specific time series analysis needs.

In [23]:
Markdown(out1[3])

Based on the insights from the previously mentioned limitations, we can formulate a research question and develop associated talking points.

Research Question:
How can the patched-decoder style attention model for time-series forecasting be improved to handle the nuanced grammar of time-series data and enhance its zero-shot generalization capabilities across various datasets with differing history lengths and prediction lengths?

Talking Points:
- **Point 1**: Addressing the lack of a defined vocabulary or grammar for time-series data by exploring the integration of domain-specific knowledge into the model's pretraining process. This could involve creating a synthetic 'language' of time-series data patterns and behaviors that the model could learn from.
  
- **Point 2**: Enhancing the model's ability to work with limited data availability by investigating techniques such as transfer learning from related time-series domains or employing data augmentation strategies to generate a more comprehensive training dataset.
  
- **Point 3**: Improving the flexibility and efficiency of the model with regards to varying input and output patch lengths by researching new model architectures that are inherently more adaptable to different time-series forecasting scenarios, possibly through dynamic attention mechanisms or modular design.
  
- **Point 4**: Developing alternative loss functions or output heads for the model that cater to both point forecasting and probabilistic forecasting, thereby expanding the model's applicability and improving its predictive performance in uncertainty quantification.

- **Point 5**: Reducing the potential for error accumulation in autoregressive decoding by experimenting with non-autoregressive approaches or error correction mechanisms that could be built into the model's inference process.

These talking points provide a framework for further research and development that could enhance the capabilities of large language model-inspired approaches to time-series forecasting, addressing current limitations and pushing the boundaries of what such models can achieve.

In [24]:
Markdown(out1[4])

Based on the information about the Multimodal Transformer (MulT) and its potential real-world applications, here is a proposed research question and associated talking points:

```json
{
  "paper": "Multimodal Transformer for Unaligned Multimodal Language Sequences",
  "summary": "The Multimodal Transformer (MulT) is designed to address the challenges of non-alignment and long-range dependencies in multimodal human language time-series data. Its directional pairwise crossmodal attention mechanism allows for the integration of unaligned multimodal sequences, leading to improved performance in multimodal tasks.",
  "research question": "How can the Multimodal Transformer (MulT) be adapted to enhance communication and learning for individuals with disabilities who rely on assistive technologies that incorporate multimodal inputs?",
  "talking points": {
    "point 1": "Investigate the efficacy of MulT in real-time assistive communication devices that interpret a combination of speech, sign language, and emotional expressions.",
    "point 2": "Explore the use of MulT in educational software that adapts to the multimodal learning patterns of students with different learning abilities, potentially improving personalized learning experiences.",
    "point 3": "Assess the impact of MulT's crossmodal attention mechanism on the accuracy of interpreting multimodal inputs in complex environments, such as noisy classrooms or public spaces for individuals with sensory impairments."
  }
}
```

These talking points are grounded in the idea that MulT's ability to process unaligned multimodal sequences without explicit data alignment could be particularly advantageous in developing assistive technologies and educational tools for individuals with disabilities. The research question is unique and practical as it seeks to extend the application of MulT to a socially impactful area, providing benefits to a demographic that often encounters barriers to effective communication and learning. The talking points support the proposed research question by outlining specific scenarios where MulT's capabilities could be transformative. If further clarification or information is needed, I will be sure to utilize the QA bot tool provided.